In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta
import os
import sys

In [ ]:
# Load custom functions
sys.path.append("./python_functions/")
from custom_data_process import twetbulb, convert_temp

In [ ]:
###############################################################################
#   Housekeeping
###############################################################################
df_temp = pd.read_csv(os.path.join("processing","Timeseries_step1_drybulb,raw_cloud,raw_relhum.csv"))
df_relhum = pd.read_csv(os.path.join("processing","Timeseries_step2_corrected_relhum.csv"))

dir_out = os.path.join("./processing")

In [ ]:
###############################################################################
#   Assign index name for convenience
###############################################################################
df_relhum.columns.values[0] = "Date"
df_relhum.columns.values[1] = "relhum"

In [ ]:
##########################################################################
# Create index in correct format using pandas date_range
##########################################################################

# Observed
dt0 = df_temp["Date"].iloc[0]
dt1 = df_temp["Date"].iloc[-1]
index = pd.date_range(dt0, dt1, freq = timedelta(hours = 1))
df_temp = df_temp.set_index(index)
df_temp = df_temp.drop("Date", axis = 1)

# Model
dt0 = df_relhum["Date"].iloc[0]
dt1 = df_relhum["Date"].iloc[-1]
index = pd.date_range(dt0, dt1, freq = timedelta(hours = 1))
df_relhum = df_relhum.set_index(index)
df_relhum = df_relhum.drop("Date", axis = 1)

In [ ]:
###############################################################################
# Compute wet-bulb temperature
###############################################################################
temp = df_temp["Dry-bulb Temperature [F]"]
relhum = df_relhum["relhum"]
twb1 = convert_temp(twetbulb(relhum/100, convert_temp(temp, "f", "c")), "c", "f")

In [ ]:
###############################################################################
# Print to csv
###############################################################################
df_print = pd.DataFrame(twb1)
df_print.index.name = "Date"
df_print.to_csv(os.path.join(dir_out, "Timeseries_step3_wetbulb.csv"),
            float_format = "%.2f", na_rep = "NaN",
            header=["Wet-bulb Temperature [F]"])